<center><h2> Generating LSST multi-band postage stamps

In this notebook, we generate samples of various sizes for the time-delay measurement project. The end result of this notebook is a csv generated with the following properties (at minimum):
* microlensing parameters
    * convergence: $\kappa$
    * stellar convergence: $\kappa_*$
    * shear: $\gamma$
* black hole accretion and transfer function parameters required by AMOEBA
    * black hole mass
    * eddington rate
    * black hole accretion disk inclination angle
* variability parameters required for a damped random walk generated using a bending power-law with fixed lower slope of 0 and higher slope of 2
    * $SF_\infty$ = $\sqrt2\sigma$ where $\sigma$ is the standard deviation of light curve variation from the mean
    * $\tau_{\rm DRW}$ where the breakpoint frequency = $1/\sqrt(2\pi\tau_{\rm DRW})$
* lensed magnitudes of 2/3/4 images
* arrival times of 2/3/4 images

### Import required pacakges

In [ ]:
### Cosmology and astropy packages
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
import astropy.coordinates as coord
import astropy.units as u

### Arrays, tables, plots
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### SLSim functions
import slsim.Sources as sources
import slsim.Deflectors as deflectors
import slsim.Pipelines as pipelines
from slsim.Sources.SourceCatalogues.QuasarCatalog.quasar_pop import QuasarRate
from slsim.Lenses.lens_pop import LensPop
from slsim.ImageSimulation.image_simulation import (
    point_source_coordinate_properties,
    lens_image_series,
)


from slsim.Util.param_util import ellipticity2phi_q

# from slsim.Util.distribution_plot_utils import make_contour
from slsim.LsstSciencePipeline.rubin_sim_pipeline import get_rubin_cadence
from lenstronomy.Util.data_util import bkg_noise


### Readin, readout, paths
from contextlib import redirect_stdout
import io

# from tqdm import tqdm

### recompile packages after each edit
%load_ext autoreload
%autoreload 2

/Users/padmavenkatraman/Documents/StrongLensing/slsim_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Set up SLSim to generate populations

In [3]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
galaxy_sky_area = Quantity(
    value=10, unit="deg2"
)  # this is the sky area over which galaxies are sampled
quasar_sky_area = Quantity(value=10, unit="deg2")

# this is the sky area over which lensed quasars are sampled
full_sky_area = Quantity(value=5000, unit="deg2")

# define limits in the intrinsic deflector and source population (in addition
# to the skypy config
# file)
kwargs_deflector_cut = {"band": "i", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "i", "band_max": 26, "z_min": 0.001, "z_max": 6.0}

In [4]:
# generate galaxy population using skypy pipeline.
galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
    skypy_config=None,
    sky_area=galaxy_sky_area,
    filters=["u", "g", "r", "i", "z", "y"],
    cosmo=cosmo,
    z_min=0,
)

### Generate lens galaxy population

In [ ]:
lens_galaxies_ell = deflectors.EllipticalLensGalaxies(
    galaxy_list=galaxy_simulation_pipeline.red_galaxies,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light={},
    cosmo=cosmo,
    sky_area=galaxy_sky_area,
    gamma_pl=dict(mean=2.0, std_dev=0.16),
)

/Users/padmavenkatraman/Documents/StrongLensing/slsim/slsim/Deflectors/DeflectorPopulation/elliptical_lens_galaxies.py:48: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  galaxy_list = param_util.catalog_with_angular_size_in_arcsec(


In [ ]:
# Initiate QuasarRate class to generate quasar sample.
quasar_class = QuasarRate(
    cosmo=cosmo,
    sky_area=quasar_sky_area,
    noise=True,
    redshifts=np.linspace(0.001, 6.00, 100),  # these redshifts are provided
    # to match general slsim redshift range in skypy pipeline.
)
# quasar sample with host galaxy
quasar_source_plus_galaxy = quasar_class.quasar_sample(
    m_min=15, m_max=28, host_galaxy=True
)

In [ ]:
# Prepare dictionary of agn variability kwargs
length_of_light_curve = 3850
MACLEOD2010_MEANS = np.array(
    [8.53308079, -23.48721021, -0.51665998, 2.28708691, 2.11640976]
)
MACLEOD2010_COV = np.array(
    [
        [0.27862905, -0.29501766, 0.00675703, 0.04606804, -0.00665875],
        [-0.29501766, 2.06855169, 0.19690851, 0.0244139, -0.29913764],
        [0.00675703, 0.19690851, 0.02785685, 0.01083628, -0.02216221],
        [0.04606804, 0.0244139, 0.01083628, 0.05636087, -0.02716507],
        [-0.00665875, -0.29913764, -0.02216221, -0.02716507, 0.3077278],
    ]
)
#############################################################################


# Prepare dictionary of agn variability kwargs
# Note: the means array and covariance matrix should be defined in following order and units:
# log(BH_mass/Msun), M_i, log(SFi_inf/mag), log(tau/days), zsrc
# variable_agn_kwarg_dict = {
#     "multivariate_gaussian_means": MACLEOD2010_MEANS,
#     "multivariate_gaussian_covs": MACLEOD2010_COV,
#     "known_band": "lsst2016-i",
# }
variable_agn_kwarg_dict = {
    "length_of_light_curve": length_of_light_curve,
    "time_resolution": 1,
    # "log_breakpoint_frequency": 1 / 20,
    # "low_frequency_slope": 1,
    # "high_frequency_slope": 3,
    # "standard_deviation": 0.9,
}
# variable_agn_kwarg_dict = {}
kwargs_quasar = {
    "variability_model": "light_curve",
    "kwargs_variability": {"agn_lightcurve", "u", "g", "r", "i", "z", "y"},
    "agn_driving_variability_model": "bending_power_law_from_distribution",
    "agn_driving_kwargs_variability": variable_agn_kwarg_dict,
    "lightcurve_time": np.linspace(0, length_of_light_curve, length_of_light_curve),
    "corona_height": 10,
    "r_resolution": 500,
}
# Initiate source population class.
source_quasar_plus_galaxies = sources.PointPlusExtendedSources(
    point_plus_extended_sources_list=quasar_source_plus_galaxy,
    cosmo=cosmo,
    sky_area=quasar_sky_area,
    kwargs_cut=kwargs_source_cut,
    list_type="astropy_table",
    catalog_type="skypy",
    point_source_type="quasar",
    extended_source_type="single_sersic",
    point_source_kwargs=kwargs_quasar,
)

In [ ]:
# Initiate LensPop class to generate lensed quasar pop.
quasar_lens_pop_ell = LensPop(
    deflector_population=lens_galaxies_ell,
    source_population=source_quasar_plus_galaxies,
    cosmo=cosmo,
    sky_area=full_sky_area,
)
image_sep = 0.5
mag_lim = 24
kwargs_lens_cuts = {
    "min_image_separation": image_sep,
    "max_image_separation": 10,
    "second_brightest_image_cut": {"i": mag_lim},
}
# drawing population
# the key difference in lens population drawing time is whether you ask for magnitude cuts or not I think?
quasar_lens_population = quasar_lens_pop_ell.draw_population(
    speed_factor=1000, kwargs_lens_cuts=kwargs_lens_cuts
)

### Make the dataframe for the population

In [ ]:
f = io.StringIO()
full_pop_df = pd.DataFrame()
with redirect_stdout(f):
    for i, lens_obj in enumerate(quasar_lens_population):
        full_pop_df = lens_obj.lens_to_dataframe(index=i, df=full_pop_df)
        image2mag = full_pop_df.loc[i, "point_source_light_i_magnitude_1"]
        try:
            image3mag = full_pop_df.loc[i, "point_source_light_i_magnitude_2"]
        except KeyError:
            image3mag = 0
        second_or_third_mag = (
            image3mag if not (np.isnan(image3mag) or image3mag == 0) else image2mag
        )
        full_pop_df.loc[i, "i3"] = second_or_third_mag
        (
            full_pop_df.loc[i, "deflector_mass_phi"],
            full_pop_df.loc[i, "deflector_mass_q"],
        ) = ellipticity2phi_q(
            full_pop_df.loc[i, "deflector_mass_e1"],
            full_pop_df.loc[i, "deflector_mass_e2"],
        )
        (
            full_pop_df.loc[i, "deflector_light_phi"],
            full_pop_df.loc[i, "deflector_light_q"],
        ) = ellipticity2phi_q(
            full_pop_df.loc[i, "deflector_light_i_e1"],
            full_pop_df.loc[i, "deflector_light_i_e2"],
        )
        full_pop_df.loc[i, "deflector_stellar_mass"] = lens_obj.deflector_stellar_mass()
        full_pop_df.loc[i, "lens_obj"] = lens_obj
        for band in list("ugrizy"):
            abs_mag = quasar_class.convert_magnitude(
                full_pop_df.loc[i, f"ps_{band}_mag_true"],
                full_pop_df.loc[i, "point_source_redshift"],
                conversion="apparent_to_absolute",
            )
            full_pop_df.loc[i, f"M_{band}"] = abs_mag
            if np.isnan(abs_mag):
                if band == "y":
                    full_pop_df.drop(index=i, inplace=True)

### Generate images

In [ ]:
def get_random_ra_dec(N=1):
    ra_points = coord.Angle(np.random.uniform(low=0, high=360, size=N) * u.degree)
    ra_points = ra_points.wrap_at(180 * u.degree)
    # dec goes from -72 to +12
    lower = -70
    upper = 10
    p = (
        np.sin(np.random.uniform(low=lower, high=upper, size=N) * u.deg)
        - np.sin(lower * u.deg)
    ) / (np.sin(upper * u.deg) - np.sin(lower * u.deg))
    dec_points = coord.Angle(
        ((((np.arcsin(2 * p - 1).to(u.deg) + 90 * u.deg) / (180 * u.deg)) * 84) + lower)
        * u.deg
    )
    return ra_points, dec_points

In [ ]:
lens_class = np.random.choice(quasar_lens_population)


# Get a point source coordinate so that you can plot these image center in the plot.
def compute_magnitude_zeropoint(mag_zp_1s, exposure_time=30, gain=1):
    return mag_zp_1s + 2.5 * np.log10(exposure_time / gain)


bands = list("ugrizy")
mag_zps = np.array(
    [
        26.52,
        28.51,
        28.36,
        28.17,
        27.78,
        26.82,
    ]
)  # taken from https://smtn-002.lsst.io/
mag_zero_points_1_second = dict(zip(bands, mag_zps))  # mag

mag_zero_points_30_seconds = dict(
    zip(bands, compute_magnitude_zeropoint(mag_zps))
)  # mag
delta_pix = 0.2  # arcsec/pixel
num_pix = 33  # pixels
exp_time = 30  # s
pix_coord_list = [
    point_source_coordinate_properties(
        lens_class,
        band=i,
        mag_zero_point=mag_zero_points_30_seconds[i],
        delta_pix=delta_pix,
        num_pix=num_pix,
        transform_pix2angle=np.array([[0.2, 0], [0, 0.2]]),
    )
    for i in bands[1:]
]
pix_coord_dict = dict(zip(bands[1:], pix_coord_list))

In [ ]:
lsst_colors = {
    "u": "#0c71ff",
    "g": "#49be61",
    "r": "#c61c00",
    "i": "#ffc200",
    "z": "#f341a2",
    "y": "#5d0000",
}

In [ ]:
# Loop through each lens, change RA and Dec, and plot the light curves
# for lens_class in random_lenses:
# Get random RA and Dec
new_ra, new_dec = get_random_ra_dec(N=1)
lens_class.ra_image = np.array([new_ra.deg])
lens_class.dec_image = np.array([new_dec.deg])

# Get Rubin cadence for the new RA and Dec
rubin_df = get_rubin_cadence(new_ra, new_dec)

observation_dates = rubin_df["observationStartMJD"]
time = np.arange(0, length_of_light_curve, 1)
image_number = lens_class.image_number
if isinstance(image_number, list):
    image_number = image_number[
        0
    ]  # taking the number of images from the first plane source

fig, ax = plt.subplots(1, image_number, figsize=(30, 7))
ax = ax.flatten()

for band in list("ugrizy"):
    time_sampled = np.array(
        observation_dates[band]
    )  # we're picking the dates that we expect to observe with Rubin
    # loop through the bands and plot the light curves
    for i in range(image_number):
        ax[i].plot(
            time,
            lens_class.point_source_magnitude(
                band=band, lensed=True, time=time, microlensing=False
            )[0][i],
            label=f"{band}-band, image-{i+1}",
            color=lsst_colors[band],
            alpha=0.2,
        )
        ax[i].scatter(
            time_sampled,
            lens_class.point_source_magnitude(
                band=band, lensed=True, time=time_sampled, microlensing=False
            )[0][i],
            marker="*",
            s=100,
            color=lsst_colors[band],
        )

ax[0].set_ylabel("Magnitude", fontsize=20)
fig.supxlabel("Time [Days]", fontsize=20)
for a in ax:
    a.invert_yaxis()
    a.legend()
fig.suptitle(
    f"Light curves for {lens_class.image_number} images of a multiply-imaged quasar",
    fontsize=30,
)
fig.tight_layout()
plt.show()

In [ ]:
transform_matrix = np.array([[delta_pix, 0], [0, delta_pix]])

# let's set up psf kernel for each exposure. Here we have taken the same psf that we
# extracted above. However, each exposure can have different psf kernel and user should
# provide corresponding psf kernel to each exposure.
psf_kernel_list = [None]
transform_matrix_list = [transform_matrix]

# psf_kernels_all = np.array([dp0["psf_kernel"][:10]])[0]

# let's set pixel to angle transform matrix. Here we have taken the same matrix for
# each exposure but user should provide corresponding transform matrix to each exposure.


# provide magnitude zero point for each exposures. Here we have taken the same magnitude
#  zero point for each exposure but user should provide the corresponding magnitude
# zero point for each exposure.
image_lens_series_all_bands = []
for band in list("grizy"):
    time_sampled = observation_dates["i"]
    # indices = np.linspace(0, len(time_sampled) - 1, 5).astype(int)
    # time_sampled = np.array([time_sampled[i] for i in indices])

    repeats = len(time_sampled)
    mag_list = [mag_zero_points_30_seconds[band]]
    psf_kernels_all = psf_kernel_list * repeats
    transform_matrix_all = transform_matrix_list * repeats
    mag_list = [mag_zero_points_30_seconds[band]]
    mag_zero_points_all = mag_list * repeats
    # mag_zero_points_all = np.array([dp0["zero_point"][:10]])[0]

    expo_list = [exp_time]
    exposure_time_all = expo_list * repeats
    image_lens_series_all = lens_image_series(
        lens_class=lens_class,
        band=band,
        mag_zero_point=mag_zero_points_all,
        num_pix=num_pix,
        psf_kernel=psf_kernels_all,
        transform_pix2angle=transform_matrix_all,
        exposure_time=exposure_time_all,
        std_gaussian_noise=bkg_noise(
            0.005, 30, np.array(rubin_df.loc["i", "skyBrightness"]), 0.2, 1
        ),
        t_obs=time_sampled,
        with_deflector=True,
        with_ps=True,
        with_source=True,
        add_noise=False,
    )
    image_lens_series_all_bands.append(image_lens_series_all)
    pix_coord = pix_coord_dict[band]["image_pix"]  # band r

    # plot_montage = create_image_montage_from_image_list(
    #     num_rows=1,
    #     num_cols=5,
    #     images=image_lens_series_all,
    #     time=time_sampled,
    #     image_center=pix_coord,
    # )
    # plot_montage.suptitle(f"{band}-band", y=1.01)

In [ ]:
# Define the bands and the number of images to plot
bands = list("ugrizy")
num_images = 10
num_rows = 5

# Select 15 equally spaced time samples
time_sampled = np.linspace(0, length_of_light_curve, num_images, dtype=int)

# Create a figure for the plot
fig, axes = plt.subplots(num_rows, num_images, figsize=(20, 15))
# axes = axes.flatten()
bands = list("grizy")
# Loop through each band and plot the images
for i, band in enumerate(bands):  # Limit to the number of rows
    for j, time_idx in enumerate(time_sampled):
        # Generate the image for the given band and time
        image = image_lens_series_all_bands[i][j]

        # Plot the image
        ax = axes[i][j]
        ax.imshow(image, origin="lower", cmap="viridis")
        ax.set_title(f"{band}-band, t={time_sampled[j]}")
        ax.axis("off")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()